# 1. 엑셀데이터 불러오기

In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re

df_input = pd.read_csv('phone_info.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

# 2. 일단 한페이지에서 파싱하기

In [83]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델영문명
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": ""                        #리뷰내용 
}
    
crawl_data = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_input.iloc[index][4]+url2)
    driver.implicitly_wait(3)
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
            pass
        else:
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") 
            market = area1[3].text 
            write_id = area1[4].text 
            upload_day = area1[5].text 
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            my_dict = {
                "pl_id": df_input.iloc[index][0],              #기종아이디(자체부여한것)
                "pl_model_code": df_input.iloc[index][3],      #모델코드
                "pl_name": df_input.iloc[index][4],            #모델영문명

                "star": star,                       #별점
                "market": market,                   #구입처(ex 인터파크, 11번가)
                "write_id": write_id,               #작성자(일부가림처리)
                "upload_day": upload_day,           #업로드날짜
                "title":title,                      #리뷰제목 
                "text": text,                       #리뷰내용 
            }
            
            if my_dict is None or my_dict == None:
                pass
            else:
                a = pd.DataFrame(my_dict,index=[0])
                crawl_data = crawl_data.append(a)
                crawl_data = crawl_data.reset_index(drop=True)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
        
    return crawl_data

# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)


# for i in range(len(df_input)):
for i in range(9,11):
    print("=================",i,"skt","=================")
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data,i,"skt")) 
    print("=================",i,"kt","=================")
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data,i,"kt"))
    print("=================",i,"lg","=================")
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data,i,"lg")) 
    

================= 9 skt =================
================= 9 kt =================
================= 9 lg =================
================= 10 skt =================
================= 10 kt =================
================= 10 lg =================


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text
0,,,,,,,,,


In [84]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
display(crawl_data)

pandas version:  1.0.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text
0,,,,,,,,,
0,,,,,,,,,
1,10,A1778,아이폰7,5,위메프,psu*****,21.03.14.,이번에 아이폰7을 좋은 기회에 무료로 바꿨어요.,이번에 아이폰7을 좋은 기회에 무료로 바꿨어요. 필름과 액정도 함께 보내주셨어요. ...
2,10,A1778,아이폰7,5,위메프,p39*****,21.03.12.,드디어 아이폰6에서 아이폰7으로 기기변경 했어요,드디어 아이폰6에서 아이폰7으로 기기변경 했어요디자인 자체는 거의 비슷해서 깔끔합니...
3,10,A1778,아이폰7,5,인터파크,myhj****,17.07.26.,간편하게 구매할 수 있어서 좋습니다,현금완납폰을 구하는데 급전은없고... 마침 딱 찾아서 현금완납 무이자 6개월 할부로...
4,10,A1778,아이폰7,5,인터파크,s*y*****,21.03.10.,넘 잘샀어요,득템했어요~! 15일 뒤에 요금 처리 잊지 말고 해 주세용~~!!^^
5,10,A1778,아이폰7,5,G마켓,v*n*****,21.03.06.,좋아요,원래 쓰던 아이폰이데 기능도 좋고 디자인도 좋고 편해요
6,10,A1778,아이폰7,5,11번가,su******,21.03.04.,최고예요,아이들 폰으로 맘에 들어요!
7,10,A1778,아이폰7,5,G마켓,k*0*****,21.03.04.,아이폰7,정말 좋습니다. ㅎㅎ
8,10,A1778,아이폰7,5,G마켓,j*j*****,21.02.21.,좋아요,초등딸아이 저렴하게 메이커 사줄수있어서 좋네요


# 불러온 엑셀데이터로 차례대로 파싱

In [35]:
 my_dict = {
    "maker": "",
    "group": "",
    "code": "",
    
    "name": "",
    "model_name": "",
    "storage": "",
    "color_name": "",
    
    "hidden":"",
    "storage_hidden": "",
    "color_hidden": "", #색상사용여부
    "class_hidden": "", #등급사용여부?
    
    "size": "",
    "display_inch": "",
    "display_cm": "",
    "weight": "",
    "battery": "",
    "front_camera": "",
    "back_camera": "",
    
    "price": "",
    "score": "", #별점
    "summary": "" #스펙요약
    }
crawl_data = pd.DataFrame(my_dict,index=[0])

In [36]:
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False

In [76]:
crawl_data.to_csv('crawl_data.csv', # file path, file name
                  sep=',')   # seperator, delimiter (구분자)
#     ...: na_rep='NaN')   # missing data representation (결측값 표기)

